In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from random import randint

In [2]:
# This parameter influences wether the visualization of the positions of bottles inside the boxes will be shown. 
# Visualization is very expensive in terms of computational power.
# DO NOT CHANGE THIS PARAMETER FOR BIG DATABASES.
# ONLY CHANGE IT IF YOU WANT A SMALL INSIGHT INTO HOW THE BOTTLES ARE PLACED.
visualize = True

In [3]:
def convert_to_needed_dataframe(old_orders, old_sku, order_number="ORDERNUMBER", 
                                                     product_article="ARTICLE_NUMBER", 
                                                     product_length="Length", 
                                                     product_width="Width", 
                                                     product_height="Height"):
    new_orders = pd.DataFrame()
    new_sku = pd.DataFrame()
    new_orders["ORDERNUMBER"] = old_orders[order_number]
    new_orders["ARTICLE_NUMBER"] = old_orders[product_article]
    new_sku["ARTICLE_NUMBER"] = old_sku[product_article]
    new_sku["Length"] = old_sku[product_length]
    new_sku["Width"] = old_sku[product_width]
    new_sku["Height"] = old_sku[product_height]
    new_orders["NUMBER_OF_BOTTLES"] = 1
    return new_orders, new_sku


def unpack_orders(df_of_orders):
    df_copy = df_of_orders.copy(deep=True)
    for order_index, order_in_list in df_of_orders.iterrows():
        if order_in_list.NUMBER_OF_BOTTLES.item() > 1:
            for i in range(order_in_list.NUMBER_OF_BOTTLES.item() - 1):
                df_copy = df_copy.append(order_in_list)
    return df_copy

Creating 3 data frames: SKU, Orders, and Box dimension.

In [4]:
df_order_original = pd.read_csv(r'data/ORDERtest.csv', encoding='utf-8-sig')
df_sku_original = pd.read_csv(r'data/SKUtest.csv', encoding='utf-8-sig')
df_order, df_sku = convert_to_needed_dataframe(df_order_original, df_sku_original, "ORDERNUMBER", "ARTICLE_NUMBER", "Length", "Width", "Height")
df_order["NUMBER_OF_BOTTLES"] = df_order_original["NUMBER_OF_BOTTLES"]
df_order = unpack_orders(df_order)
dat_box = {'Capacity': [2,6,12,'M'],
        'Length': [42,42,42, 100],
        'Width': [9.5,28.5,28.5, 100], 
        'Height': [20,20,40, 100],
        'Cost': [0.53, 1.06, 1.33, 3]}
df_box = pd.DataFrame(dat_box)
df_magnum = df_box.iloc[-1:]
df_box.drop(df_box.tail(1).index,inplace=True)
divider_cost = 0.22
df_box.head(5)

,Capacity,Length,Width,Height,Cost
0,2,42,9.5,20,0.53
1,6,42,28.5,20,1.06
2,12,42,28.5,40,1.33


Merging SKU and orders into 1 dataset, which contains the dimensions of each product.

In [5]:
df_combined = pd.merge(df_order, df_sku, how="left", on="ARTICLE_NUMBER")
df_combined.head()

,ORDERNUMBER,ARTICLE_NUMBER,NUMBER_OF_BOTTLES,Length,Width,Height
0,1,309605,3,12.0,15.0,31.5
1,1,315052,1,6.0,6.0,31.1
2,1,315052,1,6.0,6.0,31.1
3,2,309605,1,12.0,15.0,31.5
4,2,699136,1,8.3,8.3,32.0


Sorting first on *ORDERNUMBER*, then by *ARTICLE_NUMBER*. Making sure that Length is the highest dimension, height is the lowest.

In [6]:
for index, row in df_combined.iterrows():
    ar = [row.Length, row.Width, row.Height]
    ar.sort()
    df_combined.at[index, 'Length'] = ar[2]
    df_combined.at[index, 'Width'] = ar[1]
    df_combined.at[index, 'Height'] = ar[0]
df_combined['Volume'] = df_combined['Length'] * df_combined['Width'] * df_combined['Height']
df_combined.sort_values(['ORDERNUMBER', 'Volume', 'ARTICLE_NUMBER'], ascending=[True, False, True], inplace = True)

Adding column with number of bottles in an order.

In [7]:
n_of_bottles_in_order = df_combined["ORDERNUMBER"].value_counts()
n_of_bottles_in_orderFrame = pd.DataFrame(n_of_bottles_in_order)
n_of_bottles_in_orderFrame = n_of_bottles_in_orderFrame.reset_index()
n_of_bottles_in_orderFrame.columns = ['ORDERNUMBER', 'Bottles']
df_with_orders = pd.merge(df_combined, n_of_bottles_in_orderFrame, how="left", on="ORDERNUMBER")
df_with_orders.head(10)

,ORDERNUMBER,ARTICLE_NUMBER,NUMBER_OF_BOTTLES,Length,Width,Height,Volume,Bottles
0,1,309605,3,31.5,15.0,12.0,5670.00,5
1,1,309605,3,31.5,15.0,12.0,5670.00,5
2,1,309605,3,31.5,15.0,12.0,5670.00,5
3,1,315052,1,31.1,6.0,6.0,1119.60,5
4,1,315052,1,31.1,6.0,6.0,1119.60,5
5,2,309605,1,31.5,15.0,12.0,5670.00,2
6,2,699136,1,32.0,8.3,8.3,2204.48,2
7,3,670286,5,28.0,9.1,9.1,2318.68,9
8,3,670286,5,28.0,9.1,9.1,2318.68,9
9,3,670286,5,28.0,9.1,9.1,2318.68,9


The dataset to work with: **df_with_orders**.

In [8]:
cur_order = df_with_orders.loc[df_with_orders['ORDERNUMBER'] == 1]
print(cur_order)

   ORDERNUMBER  ARTICLE_NUMBER  NUMBER_OF_BOTTLES  Length  Width  Height  \
0            1          309605                  3    31.5   15.0    12.0   
1            1          309605                  3    31.5   15.0    12.0   
2            1          309605                  3    31.5   15.0    12.0   
3            1          315052                  1    31.1    6.0     6.0   
4            1          315052                  1    31.1    6.0     6.0   

   Volume  Bottles  
0  5670.0        5  
1  5670.0        5  
2  5670.0        5  
3  1119.6        5  
4  1119.6        5  


In [9]:
def cuboid_data2(o, x,y,z):
    X = [[[0, y, 0], [0, 0, 0], [x, 0, 0], [x, y, 0]],
         [[0, 0, 0], [0, 0, z], [x, 0, z], [x, 0, 0]],
         [[x, 0, z], [x, 0, 0], [x, y, 0], [x, y, z]],
         [[0, 0, z], [0, 0, 0], [0, y, 0], [0, y, z]],
         [[0, y, 0], [0, y, z], [x, y, z], [x, y, 0]],
         [[0, y, z], [0, 0, z], [x, 0, z], [x, y, z]]]
    
    X = np.array(X).astype(float)
    X += np.array(o)
    return X

def plotCubeAt2(positions, **kwargs):
    g = []
    col = []
    for p in positions:
        g.append(p)
        col.append('#%06X' % randint(0, 0xFFFFFF))
    return Poly3DCollection(np.concatenate(g),  
                            facecolors=np.repeat(col,6), **kwargs)

def all_dimensions_larger(box, bottle):
    cur_box = box.copy()
    ar = [cur_box.Length.item(), cur_box.Width.item(), cur_box.Height.item()]
    ar.sort()
    if (bottle['ARTICLE_NUMBER'].item() == 387630):
        print ("HIIIIIIIIIII!!!!!!")
    if ar[2] < bottle['Length'].item() or ar[1] < bottle['Width'].item() or ar[0] < bottle['Height'].item():
        return True
    return False


def calculate_cost(box_info, box_order, total_number_of_height_dividers):
    cost = total_number_of_height_dividers * divider_cost
    for i in box_order:
        for box_cap in box_order[i]:
            cost += box_info.loc[box_info['Capacity'] == box_cap].Cost.item()
    return cost

def number_of_height_dividers(height):
    return np.floor(height/10.00001)
    

def make_a_plot(threed_coordinates, df_box, box_for_order, current_order, visualize=False):
    if visualize:
        if threed_coordinates:
            pc = plotCubeAt2(threed_coordinates, edgecolor="k")
            ax.add_collection3d(pc)
            ax.add_collection3d(Poly3DCollection(np.concatenate([cuboid_data2([(0,0,0)], 
                                                                              df_box.loc[df_box['Capacity'] == box_for_order[current_order][len(box_for_order[current_order])-1]]['Length'].item(),
                                                                              df_box.loc[df_box['Capacity'] == box_for_order[current_order][len(box_for_order[current_order])-1]]['Width'].item(),
                                                                              df_box.loc[df_box['Capacity'] == box_for_order[current_order][len(box_for_order[current_order])-1]]['Height'].item())]),  
                                                                                color='b', alpha=0.07))
            ax.set_xlim([0,45])
            ax.set_ylim([0,45])
            ax.set_zlim([0,45])
            ax.view_init(15,-15)
            plt.show()
            plt.clf()

def get_direction(order_data, closest_box):
    direction_to_put = 'Width' if (order_data.Length > closest_box.Width.item()).any() else 'Length'
    cur_direction = closest_box[direction_to_put].item()
    const_dir = 'Width' if (cur_direction == 'Length') else 'Length'
    cur_const_dir = closest_box[const_dir].item()
    cur_height = closest_box.Height.item()
    return direction_to_put, cur_direction, const_dir, cur_const_dir, cur_height

def magnum_bottle_actions(list_of_boxes_for_current_order, index_of_current_order, current_counter, current_index):
    list_of_boxes_for_current_order[index_of_current_order].append('M')
    return current_counter - 1, current_index + 1, current_index + 1

def handling_overflow(list_of_boxes_for_current_order, index_of_current_order, size_of_box, possible_boxes_for_order,
                     dataframe_boxes, current_index, counted_dividers, new_dividers):
    list_of_boxes_for_current_order[index_of_current_order].append(size_of_box.Capacity.item())
    possible_boxes_for_order[index_of_current_order].clear()
    possible_boxes_for_order[index_of_current_order].extend(dataframe_boxes['Capacity'].unique().tolist())
    #print("OVERFLOW!!!!!!!")
    return counted_dividers + new_dividers, 0, current_index
    
    

In [1]:
divider_width = 0.5
total_number_of_dividers = 0
number_of_separator_used = 0
order_possible_boxes = {}
box_for_order = {}
for current_order in df_with_orders['ORDERNUMBER'].unique():
    order_data = df_with_orders.loc[df_with_orders['ORDERNUMBER'] == current_order]
    index = 0
    counter = 0
    heights_in_row = []
    threed_coordinates = []
    saved_index = 0
    need_divider = 0
    while (index != len(order_data.index)):
        if current_order not in box_for_order:
            box_for_order[current_order] = []
        row = order_data.iloc[index]
        if counter == 0:
            need_divider = 0
            number_of_levels_used = 0
            number_of_separator_used = 0
            dividers_on_height = 0
            if current_order not in order_possible_boxes:
                    order_possible_boxes[current_order] = df_box['Capacity'].unique().tolist()
            counter = row.Bottles - saved_index
            # The closest box capacity to the number of bottles
            #closest_box_size = order_possible_boxes[current_order][(np.abs(order_possible_boxes[current_order] - counter)).argmin()]
            closest_box_size = order_possible_boxes[current_order][0]
            closest_box = df_box.loc[df_box['Capacity'] == closest_box_size]
            
            # Determining the direction(Width or Length) in which we should put the bottles in the box
            direction_to_put, cur_direction_box_size, const_dir, cur_const_dir, cur_box_height = get_direction(order_data, closest_box)
            cur_height = 0
            cur_direction_size = 0
            
            #creating a figure
            if visualize:
                plt.clf()
                threed_coordinates.clear()
                fig = plt.figure(figsize=(10, 8))
                ax = fig.gca(projection='3d')
                ax.set_aspect('auto')
            
                        
        #print('Order: ', current_order, ' Item: ', index+1,'Current box size: ', closest_box_size, 'Current length: ', cur_direction_size, ' Current height: ', cur_height)
        
        # Check if the magnum bottle found
        if (all_dimensions_larger(df_box.iloc[-1:], row)):
            counter, index, saved_index = magnum_bottle_actions(box_for_order, current_order, counter, index)
            continue

                        
        # Check if there is still a place in height to place the bottle inside the box
        #print(cur_height, ' Row height ', row.Height.item())
        if cur_height + row.Height.item() <= cur_box_height:
            # Check if there is still a place in width/length to place the bottle inside the box
            if cur_direction_size + row['Width'].item() <= cur_direction_box_size:
                # If there is place in width - place a bottle
                threed_coordinates.append(cuboid_data2([(closest_box[const_dir].item() - cur_const_dir,
                                                        cur_direction_size + need_divider,
                                                        cur_height)], 
                                                    row['Length'].item(),row['Width'].item(),row['Height'].item()))
                cur_direction_size += row['Width'].item() + need_divider
                #print("I clace a separator", number_of_separator_used)
                heights_in_row.append(row['Height'].item())
                if need_divider == 0:
                    need_divider = divider_width
                else:
                    dividers_on_height += 1
                #print("I placed a bootle!")
            else:
                # If there is no place in width - start new row on top of the current
                if len(heights_in_row) != 0:
                    cur_height += (max(heights_in_row) + divider_width)
                else:
                    cur_height += (row.Height.item() + divider_width)
                if (number_of_levels_used < number_of_height_dividers(cur_box_height)):
                    number_of_levels_used += 1
                number_of_separator_used += np.ceil(dividers_on_height/2.0)
                dividers_on_height = 0
                need_divider = 0
                heights_in_row.clear()
                cur_direction_size = 0
                #print("I increased a height")
                continue
        else:
            # If there is no place in height, that means that the box is not sutable for the order
            # Now check if we are inpecting the largest box
            if (len(order_possible_boxes[current_order]) == 1):
                # If we are and order doesn't fit into this largest box, that means we need extra box for the order
                total_number_of_dividers ,counter, saved_index = handling_overflow(box_for_order, current_order, 
                                                                closest_box, order_possible_boxes, df_box, 
                                                                index, total_number_of_dividers, 
                                                                number_of_levels_used+number_of_separator_used)
                make_a_plot(threed_coordinates, df_box, box_for_order, current_order, visualize)
                continue
            else:
                # If the current box is not the largest, we choose the next box dimension
                index = saved_index
                #print("Index ", index)
                counter = 0
                order_possible_boxes[current_order] = list(np.setdiff1d(order_possible_boxes[current_order], 
                                                                 np.array([closest_box.Capacity.item()])))
                continue
        #print('Order: ', current_order, ' Item: ', index+1,'Current box size: ', closest_box_size, 'Current length: ', cur_direction_box_size, ' Current height: ', cur_box_height)
        counter -= 1
        if (counter == 0):
            number_of_separator_used += np.ceil(dividers_on_height/2.0)
            #print("Number of vert dividers:", number_of_separator_used ,number_of_separator_used)
            total_number_of_dividers += (number_of_levels_used + number_of_separator_used)
            #print('Number of dividers', total_number_of_dividers)
            box_for_order[current_order].append(closest_box.Capacity.item())
        index += 1
    #ax.set_aspect('equal')
    #print(box_for_order)
    make_a_plot(threed_coordinates, df_box, box_for_order, current_order, visualize)
df_box_with_magnum = df_box.append(df_magnum)
print(box_for_order)
print(total_number_of_dividers)
print("Cost: ", calculate_cost(df_box_with_magnum, box_for_order, total_number_of_dividers))

NameError: name 'df_with_orders' is not defined